In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

from qiskit_metal.qlibrary.tlines.mixed_path import RouteMeander, RouteMixed
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

# from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond


import pyEPR as epr

from init_notebook import *

from qiskit_metal.toolbox_metal.parsing import *
from collections import OrderedDict

In [3]:
import pandas as pd
import numpy as np

### Function that generates anchors for CPW. 

Currently only used to calculate the location of the short to ground component. The RouteMixed or RouteAnchors function does not work

In [4]:
def s_anchors(TQ_options, cpw_options,flip_y = False):
    if flip_y:
        factor = -1
    else:
        factor = 1
    x0 = parse_value('pos_x',TQ_options)
    y0 = parse_value('pos_y',TQ_options)
    prime_gap = parse_value('prime_gap',TQ_options)
    prime_pin = parse_value('prime_width',TQ_options)
    coupling_space = parse_value('coupling_space',TQ_options)
    down_len = parse_value('down_length',TQ_options)

    fillet = parse_value('fillet',cpw_options)
    total_len = parse_value('total_length',cpw_options)

    l = (total_len  - fillet*np.pi)/3

    angle = parse_value('orientation',TQ_options)
    angle = np.deg2rad(angle)

    # print(x0,prime_pin,prime_gap,coupling_space,down_len,l,fillet)
    # d = (prime_pin/2+prime_gap+coupling_space)#-down_len)

    x0 = x0 + (prime_pin/2+prime_gap+coupling_space+down_len)*np.sin(angle)
    y0 = y0 - (prime_pin/2+prime_gap+coupling_space+down_len)*np.cos(angle)*factor

    x1 = x0 + l*np.sin(angle)
    y1 = y0 - l*np.cos(angle)*factor

    x2 = x1 + fillet*np.cos(angle)
    y2 = y1 + fillet*np.sin(angle)*factor

    x3 = x2 - l*np.sin(angle)
    y3 = y2 + l*np.cos(angle)*factor

    x4 = x3 + fillet*np.cos(angle)
    y4 = y3 + fillet*np.sin(angle)*factor

    x5 = x4 + l*np.sin(angle)
    y5 = y4 + l*np.cos(angle)*factor

    result = OrderedDict()
    result[0] = [x1,y1]
    result[1] = [x2,y2]
    result[2] = [x3,y3]
    result[3] = [x4,y4]
    result[4] = [x5,y5]
    return result


# Initialize things by directly importing

In [5]:
gui = MetalGUI(design)
design = design
design.overwrite_enabled = False #This makes it possible to edit the designs later on. 

In [6]:
design.overwrite_enabled = True

In [7]:
design.delete_all_components()

In [8]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60

## Specify resonator specs

Hand measured using kLayout

In [9]:
np.full((8), 350)

array([350, 350, 350, 350, 350, 350, 350, 350])

In [10]:
# coupling_d = [100,105,140,200,2,15,40,75] # low coupling
coupling_d = np.array([100,105,140,200,2,15,40,75])/2 # increased coupling 
curve_d = np.full((8), 450)#[380,380,380,400,300,310,330,340]
ground_plane_d = 10
fillets = np.full((8), 320.0)#[200,200,280,320,200,200,200,240]

prime_pin = 14.18
prime_gap = 8

res_orientation = [(-1)**i*90 for i in range(8)]
res_flip_y = [i%2==0 for i in range(8)]

res_order = [4,2,6,0, 7,1,5,3]


# Put down wirebond pads

In [11]:
wb_x0 = 4.7/2
wb_y0 = 3.5-1.86

In [12]:
Wirebond_opt = Dict(trace_width=design.variables['trace_width'],
                           trace_gap=design.variables['trace_gap'],
                           lead_length='440.5um',
                           pad_width='200um',
                           pad_height='500um',
                           pad_gap='100um',
                           taper_height='100um',
                           layer = str(qubit_layer),
                           pin_space = '200um')

In [13]:
left_lauch_pads = LaunchpadWirebondCustom(design,'wb_left', options = Dict(pos_x = '{}mm'.format(-wb_x0),pos_y = '{}mm'.format(-wb_y0),orientation = '90',**Wirebond_opt))
right_lauch_pads = LaunchpadWirebondCustom(design,'wb_right', options = Dict(pos_x = '{}mm'.format(wb_x0),pos_y = '{}mm'.format(-wb_y0),orientation = '90',**Wirebond_opt))

gui.rebuild()
gui.autoscale()


# Put Down Resonator with Coupled Line Tee

Import resonator parameters. Calculated using coupler_optimization.ipynb with website to optimize frequency and impedance. All the other parameters are measured from dxf file

In [14]:
resonator_params = pd.read_csv('results_more_coupling.csv')
# resonator_params['coupling_gap'] = resonatlingor_params['coupling_gap']/2
# resonator_params.to_csv('results_more_coup.csv')
resonator_params['Gap'] *= 2
resonator_params['Pin'] *= 2
resonator_params['coupling_gap'] *= 2
print(resonator_params)



   Unnamed: 0.1  Unnamed: 0  Frequency          Q  Length  Gap    Pin  \
0             0           0        5.9  1278612.0  5.0535   20  35.46   
1             1           1        6.2  1210136.0  4.8072   24  42.54   
2             2           2        6.5  1149617.0  4.5836   28  49.64   
3             3           3        6.7  1113073.0  4.4457   32  56.74   
4             4           4        7.0  1063157.0  4.2536    4   7.10   
5             5           5        7.2  1032764.0  4.1344    8  14.18   
6             6           6        7.5   990927.0  3.9674   12  21.28   
7             7           7        7.9   941290.0  3.7646   16  28.36   

   coupling_gap  
0         118.0  
1         123.0  
2         158.0  
3         158.0  
4          20.0  
5          33.0  
6          58.0  
7          93.0  


In [15]:
import copy

In [16]:
series = copy.copy(resonator_params.iloc[-1])
gap1_old = series['Gap']
coupling1_old = series['coupling_gap']
series_2 = copy.copy(resonator_params.iloc[-2])
gap2_old = series_2['Gap']
coupling2_old = series_2['coupling_gap']

In [17]:
series['Pin'] = 3
series['Gap'] = 3
series['coupling_gap'] = 3/gap1_old*coupling1_old



series_2['Pin'] = 30
series_2['Gap'] = 30
series_2['coupling_gap'] = 30/gap2_old*coupling2_old


In [18]:
resonator_params = resonator_params.iloc[:-2]

In [19]:
resonator_params = resonator_params.append(series_2)
resonator_params = resonator_params.append(series)


  resonator_params = resonator_params.append(series_2)

  resonator_params = resonator_params.append(series)



In [20]:
resonator_params.to_csv('results_more_coupling_big_gap.csv')
resonator_params

,Unnamed: 0.1,Unnamed: 0,Frequency,Q,Length,Gap,Pin,coupling_gap
0,0.0,0.0,5.9,1278612.0,5.0535,20.0,35.46,118.0000
1,1.0,1.0,6.2,1210136.0,4.8072,24.0,42.54,123.0000
2,2.0,2.0,6.5,1149617.0,4.5836,28.0,49.64,158.0000
3,3.0,3.0,6.7,1113073.0,4.4457,32.0,56.74,158.0000
4,4.0,4.0,7.0,1063157.0,4.2536,4.0,7.10,20.0000
5,5.0,5.0,7.2,1032764.0,4.1344,8.0,14.18,33.0000
6,6.0,6.0,7.5,990927.0,3.9674,30.0,30.00,145.0000
7,7.0,7.0,7.9,941290.0,3.7646,3.0,3.00,17.4375


In [21]:
freqs = resonator_params['Frequency'].values
Qs = resonator_params['Q'].values
gaps = resonator_params['Gap'].values
pins = resonator_params['Pin'].values
coupling_gap = resonator_params['coupling_gap'].values
lengths = resonator_params['Length'].values

In [22]:
coupling_gap

array([118.    , 123.    , 158.    , 158.    ,  20.    ,  33.    ,
       145.    ,  17.4375])

Define locations of resonators by defining location of the coupled line tee

In [23]:
res_x = 1.15
delta_x = 3.5-res_x

res_x_all  = np.array([-delta_x*(-1)**i for i in range(8)])

res_y_all = np.array([(5254+5210)/2,(4301+4248)/2,(3448+3387)/2,
             (2395+2325)/2,(2365+2356)/2,(3326+3309)/2,
             (4338+4311)/2,(5249+5214)/2])/1000-3.5

pre define some options

In [24]:
TQ_options = TQ_options
TQ_options['prime_length'] = '120um'
TQ_options['prime_gap'] = design.variables['trace_gap']
TQ_options['prime_width'] = design.variables['trace_width']

## Actual code that puts down CPW and CoupledLineTee

NOTE: need to fix code and use RouteMixed with anchors to route CPWs. Currently using RouteMeander and lack control. 

In [25]:
CPW_options['lead']

{'start_straight': '5um', 'end_straight': '5um'}

In [26]:
TQs = []
short_to_grounds = []
cpws = []

connection_cpws = []

for i in range(8):
    curve_d_i = curve_d[i]
    coupling_d_i = coupling_d[i]
    pins_i = pins[i]
    gap_i = gaps[i]
    fillet = fillets[i]

    down_len =  - prime_pin/2 - prime_gap + curve_d_i+fillet*np.pi/2

    TQ_options['down_length'] =  str(down_len) + 'um'
    TQ_options['coupling_space'] = str(ground_plane_d) + 'um'
    TQ_options['second_width'] = str(pins[i]) + 'um'
    TQ_options['second_gap'] = str(gaps[i]) + 'um'
    # TQ_options['coupling_space'] = str(coupling_gap[i])+ 'um'
    TQ_options['pin_drop'] = str(coupling_d[i])+'um'
    TQ_options['orientation'] = str(res_orientation[i])


    TQs.append(CoupledLineTee(design, 'TQ_'+str(i), options=dict(pos_x='{} mm'.format(res_x_all[i]),
                                                pos_y='{} mm'.format(res_y_all[i]), **TQ_options)))

    CPW_options = CPW_options
    CPW_options['pin_width'] = str(pins[i]) + 'um'
    CPW_options['gap_width'] = str(gaps[i]) + 'um'
    CPW_options['total_length'] = str(np.around(lengths[i]-down_len/1000,4)) + 'mm'
    CPW_options['pin_inputs'] = in_inputs=Dict(
                start_pin=Dict(component='TQ_'+str(i), pin='second_end'),
                end_pin=Dict(component='short'+str(i), pin='short'),)

    CPW_options['trace_gap'] = str(gaps[i]) + 'um'
    CPW_options['trace_width'] = str(pins[i]) + 'um'
    CPW_options['lead'] = dict(start_straight = '600 um', end_straight = '400 um')
    CPW_options['fillet'] = '{} um'.format(fillet)
    CPW_options['prevent_short_edges'] = False
    CPW_options['snap'] = True

    anchors = s_anchors(TQs[-1].options, CPW_options,flip_y=res_flip_y[i])
    orientation = parse_value('orientation', TQ_options) + 270
    short_to_grounds.append(ShortToGround(design, 'short'+str(i), options=dict(pos_x='{} mm'.format(anchors[4][0]), pos_y='{} mm'.format(anchors[4][1]), 
                                                        orientation='{}'.format(orientation))))
 
    CPW_options['fillet'] = '{} um'.format(fillet/2.2)
    CPW_options['anchor'] = anchors
    CPW_options['meander'] = dict(spacing= '{}um'.format(fillet), asymmetry= '0um')
    CPW_options['lead'] = {'start_straight': '600 um', 'end_straight': '100 um'}

    cpws.append(RouteMeander(design, 'cpw'+str(i), options = CPW_options))
                
    

    gui.rebuild()
    gui.autoscale()

### Connect coupled line tees

In [27]:
for i, cpw in enumerate(res_order):
        if i ==7:
                break
        elif i == 3:
                continue
        else:
                trans_ops = trans_options
                trans_ops['pin_inputs'] = {'start_pin': {'component': TQs[cpw].name, 
                                                        'pin': 'prime_end'},
                                        'end_pin': {'component': TQs[res_order[i+1]].name, 
                                                        'pin': 'prime_start'}}
                connection_cpws.append(RouteStraight(design, 
                                                'connection_cpw'+str(cpw), 
                                                options = Dict(**trans_options)))
                gui.rebuild()

In [28]:
anchors = OrderedDict()
anchors[0] = (3.5-1.230,-0.2)
anchors[1] = (3.5-1.230,0.2)

In [29]:
i = 3
trans_ops1 = trans_options
trans_ops1['lead'] = dict(start_straight = '300 um', end_straight = '300 um')
trans_ops1['pin_inputs'] = {'start_pin': {'component': TQs[res_order[i]].name, 'pin': 'prime_end'},
                           'end_pin': {'component': TQs[res_order[i+1]].name, 'pin': 'prime_start'}}
trans_ops1['fillet'] = '90um'
total_len = 0.3*2+4.7 + 0.9*(np.pi-2)
trans_ops1['total_length'] = '{}mm'.format(total_len)
cpw_top = RouteMixed(design, 'connection_cpw'+str(i),options = Dict(anchor = anchors,**trans_ops1))

gui.rebuild()
                

The ideal way is to connect the Tee with launcher pad using a CPW, but this thing does not export properly so I used a skechy fix. Currently working fine.

In [30]:
# connection_cpw_ops = {'trace_width': '14.18 um',
#  'trace_gap': '8 um',
#  'lead': {'start_straight': '300 um', 'end_straight': '300 um'},
#  'layer': 5,
#  'hfss_wire_bonds': False,
#  'q3d_wirebonds': False,
#  }
# connection_cpw_ops['pin_inputs'] = {'end_pin': {'component':'TQ_4','pin': 'prime_start'},'start_pin': {'component': 'wb_left', 'pin': 'tie'}}
# # connection_cpws.append(RouteStraight(design,'connection_cpw'+str(cpw), options = Dict(**trans_options)))

# cpw_left_launch = RouteStraight(design, 'cpw_left_launch', options = Dict(**connection_cpw_ops))

# connection_cpw_ops['pin_inputs'] = {'end_pin': {'component':'TQ_3','pin': 'prime_end'},'start_pin': {'component': 'wb_right', 'pin': 'tie'}}
# cpw_right_launch = RouteStraight(design, 'cpw_right_launch', options = Dict(**connection_cpw_ops))

# gui.rebuild()

In [31]:
# design.chips.main.size['size_x'] = '10mm'
# design.chips.main.size['size_y'] = '10mm'
# gui.rebuild()
a_gds = design.renderers.gds

In [32]:
qubit_layer = 5
junction_layer = 20
ab_layer = 31
ab_square_layer = 30
junction_area_layer = 60

In [33]:
a_gds.options['max_points'] = 3000
a_gds.options['fabricate'] = True

In [34]:
a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
a_gds.options['cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = False
a_gds.options['cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = False
a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = False
a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
a_gds.options['cheese']['view_in_file']['main'][1] = False
a_gds.options['no_cheese']['view_in_file']['main'][1] = False

In [35]:
# a_gds.options['cheese']['view_in_file']['main'][qubit_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][qubit_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][junction_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][junction_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][ab_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][ab_layer] = True
# a_gds.options['cheese']['view_in_file']['main'][ab_square_layer] = True
# a_gds.options['no_cheese']['view_in_file']['main'][ab_square_layer] = True
# a_gds.options['cheese']['view_in_file']['main']['1'] = True
# a_gds.options['no_cheese']['view_in_file']['main']['1'] = True



# a_gds.options['fabricate'] = False
a_gds.options.tolerance = '0.0000005'
a_gds.options.precision = '0.0000000005'

In [36]:
a_gds.options['cheese']

{'datatype': '100',
 'shape': '0',
 'cheese_0_x': '25um',
 'cheese_0_y': '25um',
 'cheese_1_radius': '100um',
 'view_in_file': {'main': {1: False,
   5: True,
   20: False,
   31: False,
   30: False}},
 'delta_x': '100um',
 'delta_y': '100um',
 'edge_nocheese': '200um'}

In [37]:
a_gds.options.cheese.edge_nocheese = '150um'
a_gds.options.no_cheese.buffer = '200um'
a_gds.options.cheese.cheese_1_radius = '100um'
a_gds.options['negative_mask'] = dict(main= [qubit_layer])

In [38]:
# gui.rebuild()
a_gds.export_to_gds('coupled_resonator_design_big_fat.gds')

04:01PM 23s WARNING [_import_junction_gds_file]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"c:\Users\slab\Desktop\Wendy-qiskit-code\Projects\resources".


1